https://github.com/Huffon/klue-transformers-tutorial/blob/master/natural_language_inference.ipynb

- klue-roberta
https://colab.research.google.com/drive/1ymEDr6Bz2HqN0xd5plSSntYhaXlCv7ar#scrollTo=8nxObhbyw3Bn&line=1&uniqifier=1

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.4 MB/s eta 0:00:00


In [ ]:
# numpy, pandas, sklearn 등 필요한 라이브러리 import
import sys
import numpy as np
import pandas as pd
import random
from tqdm.notebook import tqdm
import os
import re

import transformers
from transformers import BertTokenizer,AdamWeightDecay,TFRobertaModel,TFBertModel

import tensorflow as tf
import keras
from keras.callbacks import EarlyStopping,ModelCheckpoint

import sklearn
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import StratifiedKFold

In [ ]:
#GPU 확인을 위한 이름 출력력
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

df_gas = pd.read_csv("/content/drive/MyDrive/NLPicasso/데이터 전처리/가스라이팅 증강.csv")
df_chatbot = pd.read_csv("/content/drive/MyDrive/NLPicasso/데이터 전처리/ChatbotData.csv")
df_gas.columns = ["prompt", "competition",	"label"]
df_chatbot.columns = ["prompt", "competition",	"label"]

df_gas['label'].fillna(1, inplace=True)
df_chatbot['label'] = df_chatbot['label'].replace([1, 2], 0)

df = pd.concat([df_gas, df_chatbot], axis=0, ignore_index=True)
df.drop(['prompt'], axis=1, inplace=True)
df = df[df['label'] != ',']
df['label'] = df['label'].astype(int)
df_all = df.sample(frac=1).reset_index(drop = True)

In [ ]:
df_all['competition'].nunique(), df_all['label'].nunique() # reivews 에 1개 중복값 발견
df_all[df_all.duplicated(subset='competition', keep=False)] # 중복값 확인
df_all.drop_duplicates(subset=['competition'], inplace=True) #중복값 제거
df_all.info()                                            #데이터확인
df_all["label"].unique()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9477 entries, 0 to 15216
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   competition  9477 non-null   object
 1   label        9477 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 222.1+ KB


array([1, 0])

In [ ]:
# Kobert 모델과 Klue/roberta 모델 중 Klue/roberta 모델이 성능이 뛰어난것을 확인
tokenizer = BertTokenizer.from_pretrained('klue/roberta-large')

In [ ]:
# 그래프 표기 관련
def reset_seeds(seed, reset_graph_with_backend=None):
    if reset_graph_with_backend is not None:
        K = reset_graph_with_backend
        K.clear_session()
        tf.compat.v1.reset_default_graph()
        print("KERAS AND TENSORFLOW GRAPHS RESET")

    np.random.seed(seed)
    random.seed(seed+100)
    tf.compat.v1.set_random_seed(seed+200)
    os.environ['CUDA_VISIBLE_DEVICES'] = ''
    print("RANDOM SEEDS RESET {}".format(seed))

SEED = 1514
reset_seeds(SEED)

RANDOM SEEDS RESET 1514


In [ ]:
# Cross validation, StratifiedKfold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds=[]

for train_idx, valid_idx in skf.split(df_all, df_all['label']):
    train_idx = np.array(list(set(list(train_idx))))
    valid_idx = np.array(list(set(set(valid_idx))))
    folds.append((train_idx, valid_idx))

In [ ]:
# klue/roberta-large 모델 tokenizer 로 train, valid, test 데이터를 정수로 encoding
def convert_data(data_df,case,mask_token):
    global tokenizer

    tokens, masks, segments, targets = [], [], [], []

    for i in tqdm(range(len(data_df))):
        # tokenize
        token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length=SEQ_LEN, padding='max_length',truncation=True)

        # making input mask
        num_zeros = token.count(mask_token)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros

        # making segment
        segment = [0]*SEQ_LEN

        # token, mask, segment
        tokens.append(token)
        masks.append(mask)
        segments.append(segment)

        if case=='train':
          # label values
          targets.append(data_df[LABEL_COLUMN][i])

    # convert to array format
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    if case=='train':
      targets = np.array(targets)

    if case=='train':
       return [tokens, masks, segments], targets
    if case=='test':
       return [tokens, masks, segments]


# Load data and convert to bert input format
def load_data(pandas_dataframe,case,mask_token):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    if case=='train':
      data_df[LABEL_COLUMN] = data_df[LABEL_COLUMN].astype(int)
      data_x, data_y = convert_data(data_df,'train',mask_token)
      return data_x, data_y
    if case=='test':
      data_x = convert_data(data_df,'test',mask_token)
      return data_x

# Define max_len
SEQ_LEN = 30
DATA_COLUMN = "competition"
LABEL_COLUMN = "label"

# train
train_x0, train_y0 = load_data(df_all.iloc[folds[0][0]].reset_index(drop=True),'train',1)
train_x1, train_y1 = load_data(df_all.iloc[folds[1][0]].reset_index(drop=True),'train',1)
train_x2, train_y2 = load_data(df_all.iloc[folds[2][0]].reset_index(drop=True),'train',1)
train_x3, train_y3 = load_data(df_all.iloc[folds[3][0]].reset_index(drop=True),'train',1)
train_x4, train_y4 = load_data(df_all.iloc[folds[4][0]].reset_index(drop=True),'train',1)

valid_x0, valid_y0 = load_data(df_all.iloc[folds[0][1]].reset_index(drop=True),'train',1)
valid_x1, valid_y1 = load_data(df_all.iloc[folds[1][1]].reset_index(drop=True),'train',1)
valid_x2, valid_y2 = load_data(df_all.iloc[folds[2][1]].reset_index(drop=True),'train',1)
valid_x3, valid_y3 = load_data(df_all.iloc[folds[3][1]].reset_index(drop=True),'train',1)
valid_x4, valid_y4 = load_data(df_all.iloc[folds[4][1]].reset_index(drop=True),'train',1)


  0%|          | 0/7581 [00:00<?, ?it/s]

  0%|          | 0/7581 [00:00<?, ?it/s]

  0%|          | 0/7582 [00:00<?, ?it/s]

  0%|          | 0/7582 [00:00<?, ?it/s]

  0%|          | 0/7582 [00:00<?, ?it/s]

  0%|          | 0/1896 [00:00<?, ?it/s]

  0%|          | 0/1896 [00:00<?, ?it/s]

  0%|          | 0/1895 [00:00<?, ?it/s]

  0%|          | 0/1895 [00:00<?, ?it/s]

  0%|          | 0/1895 [00:00<?, ?it/s]

In [ ]:
class Klue_RobertaClassifier(tf.keras.Model):
    def __init__(self, num_class):
        super(Klue_RobertaClassifier, self).__init__()

        self.bert = TFRobertaModel.from_pretrained("klue/roberta-large", from_pt=True)
        self.dropout = tf.keras.layers.Dropout(self.bert.config.hidden_dropout_prob)
        self.classifier = tf.keras.layers.Dense(num_class, kernel_initializer=tf.keras.initializers.TruncatedNormal(self.bert.config.initializer_range,seed=42),
                                                name="classifier")

    def call(self, inputs, attention_mask=None, token_type_ids=None, training=False):

        # outputs value : sequence_output, pooled_output, (hidden_states), (attentions)
        outputs = self.bert(inputs, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output, training=training)
        logits = self.classifier(pooled_output)

        return logits

klue_roberta_model = Klue_RobertaClassifier(num_class=2)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.decoder.bias', 'lm_head.bias', 'roberta.embeddings.position_ids', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream 

In [ ]:
for i in range(2):
  print('########## Fold {} : \n'.format(i))

  klue_roberta_model = Klue_RobertaClassifier(num_class=4)

  # Defining loss function, optimizer and metric
  optimizer = AdamWeightDecay(1e-5,weight_decay_rate=1e-4)
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
  klue_roberta_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

  # Adding an ealrystop to prevent overfitting
  earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001,patience=2)

  checkpoint_path = os.path.join('/content/drive/MyDrive/Colab/Contest/model/','weight_klue_roberta_back_skf_fold_v0{}.h5'.format(i))
  cp_callback = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True)

  # Training
  history = klue_roberta_model.fit(globals()['train_x{}'.format(i)],globals()['train_y{}'.format(i)], epochs=3, batch_size=32,
                             validation_data=(globals()['valid_x{}'.format(i)],globals()['valid_y{}'.format(i)]), callbacks=[earlystop_callback, cp_callback])

  klue_roberta_model.load_weights('/content/drive/MyDrive/Colab/Contest/model/'+'weight_klue_roberta_back_skf_fold_v0{}.h5'.format(i))

  preds=tf.argmax(klue_roberta_model.predict(globals()['valid_x{}'.format(i)]),axis=1)

  print('Validation set ACC: ',accuracy_score(globals()['valid_y{}'.format(i)],preds))
  print('Validation set Confusion Matrix: \n',confusion_matrix(globals()['valid_y{}'.format(i)],preds))

########## Fold 0 : 



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.decoder.bias', 'lm_head.bias', 'roberta.embeddings.position_ids', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream 

Epoch 1/3
237/237 [==============================] - ETA: 0s - loss: 0.0812 - accuracy: 0.9689
Epoch 1: val_accuracy improved from -inf to 0.99789, saving model to /content/drive/MyDrive/Colab/Contest/model/weight_klue_roberta_back_skf_fold_v00.h5


In [ ]:
def sentence_predict(sent, model, tokenizer):
    # 토크나이징
    tokenized_sent = tokenizer.encode_plus(
        sent,
        return_tensors="tf",
        truncation=True,
        add_special_tokens=True,
        max_length=128,
        pad_to_max_length=True
    )

    # 예측
    logits = model.predict([tokenized_sent["input_ids"], tokenized_sent["attention_mask"], tokenized_sent["token_type_ids"]])
    result = np.argmax(logits, axis=-1)[0]

    # 결과 return
    if result == 1:
        return " >> 가스라이팅 👿"
    elif result == 0:
        return " >> 정상 😀"

# 사용자 입력 및 예측 결과 출력
def predict_interactively(model, tokenizer):
    while True:
        sentence = input("댓글을 입력해주세요: ")
        if sentence == "0":
            break
        print(sentence + sentence_predict(sentence, model, tokenizer))
        print("\n")

# Example: KLUE Roberta 모델을 사용하여 사용자 입력을 기반으로 예측 수행
# 모델을 불러올 때 체크포인트 파일 경로를 제공해야 합니다.
# 여기서는 예시로 첫 번째 fold의 모델을 사용하였습니다.
klue_roberta_model.load_weights('/content/drive/MyDrive/Colab/Contest/model/weight_klue_roberta_back_skf_fold_v00.h5')
predict_interactively(klue_roberta_model, tokenizer)
